# 2.3 Textlicher Teil Bebauungsplan – OCR (GPT-4o)

* Zero-Shot
* Chain-Of-Thought
* Best result: A2

In [44]:
from utils.pdf2prompts import pdf2prompts
import os
import asyncio
from dotenv import load_dotenv, find_dotenv
from langchain_openai import ChatOpenAI
from langchain.schema.messages import AIMessage, HumanMessage

load_dotenv(find_dotenv())

model = ChatOpenAI(
    openai_api_key=os.getenv("OPENAI_API_KEY"),
    model_name="gpt-4o",
    model_kwargs={"top_p": 0, "seed": 42},
    temperature=0,
)

In [25]:
# pdf_path = "../data/raw/bpläne/2_zeichnung_textteil_getrennt/F 11- 02 Gewerbegebiet Himmelreich - schriftlicher Teil.pdf"
pdf_path = "../data/raw/bpläne/3_xplanung/BP_872A_AUFST_TT.pdf"
pdf_prompts = pdf2prompts(pdf_path)
print(len(pdf_prompts))

66


In [35]:
# A.1) Ein Thread – Analyse und Extraktion in einem Schritt
# Problem: Kontext ist sehr groß, sodass GPT den Überblick verliert und Informationen verliert. 
msg = model.invoke([
        AIMessage(
            content=[
                {
                    "type": "text",
                    "text": "Du bist ein Assistent zur getreuen Wiedergabe von Informationen aus einem Bebauungsplan. Achte auf Vollständigkeit."
                },
            ]
        ),
        HumanMessage(
            content=[
                {
                    "type": "text",
                    "text": 'Extrahiere alle Informationen zu den folgenden Themen: Art der baulichen Nutzung, Maß der baulichen Nutzung, Bauweise und überbaubare Grundstücksfläche. Liste am Ende die jeweiligen Seitenzahlen als Referenz auf. Output-Format: ###<Thema>: <Informationen> ### Referenzen: <Seitenzahlen>)'
                    # Optional: Zusätzlich Dach, Garage und Stellplatz
                },
                *pdf_prompts,
            ]
        ),
    ])
print(msg.content)

### Art der baulichen Nutzung:
Die in der Planzeichnung (Teil A) mit WA1 bis WA4 gekennzeichneten Bereiche werden als Allgemeines Wohngebiet gemäß § 4 BauNVO festgesetzt.

Zulässig sind:
- Wohngebäude
- Läden, Schank- und Speisewirtschaften sowie nicht störende Handwerksbetriebe zur Versorgung des Gebietes
- Anlagen für kirchliche, kulturelle, soziale, gesundheitliche und sportliche Zwecke

Nicht zulässig sind:
- Betriebe des Beherbergungsgewerbes
- sonstige nicht störende Gewerbebetriebe
- Anlagen für Verwaltungen
- Gartenbaubetriebe
- Tankstellen

(Seite 6)

### Maß der baulichen Nutzung:
Das Maß der baulichen Nutzung ist in den Nutzungsschablonen in der Planzeichnung (Teil A) festgesetzt.

In den allgemeinen Wohngebieten darf die zulässige Grundflächenzahl (GRZ) durch die Grundflächen der in § 19 Abs. 4 BauNVO aufgeführten Anlagen jeweils bis zu folgender GRZ überschritten werden:
- WA1: GRZ maximal 0,65
- WA2: GRZ maximal 0,80
- WA3: GRZ maximal 0,75
- WA4: GRZ maximal 0,75

(Seite

In [45]:
# A.2) Ein Thread – Relevante Seiten extrahieren (Vorverarbeitung) – (Bestes Ergebnis)
# Wenn Dokument zu groß wird: z.B. chunk_size=50 pages, overlap=10 pages.
msg1 = model.invoke([
        AIMessage(
            content=[
                {
                    "type": "text",
                    "text": "Du bist ein Assistent zur getreuen Wiedergabe von Informationen aus einem PDF Dokument. Achte auf Vollständigkeit."
                },
            ]
        ),
        HumanMessage(
            content=[
                {
                    "type": "text",
                    "text": 'Die Bilder repräsentieren ein PDF Dokument. Extrahiere alle Seiten zu den folgenden Themen: Art der baulichen Nutzung, Maß der baulichen Nutzung, Bauweise und überbaubare Grundstücksfläche. JSON-Output-Format: {"Seiten": [<Seitenzahlen>]}'
                },
                *pdf_prompts,
            ]
        ),
    ])
print(msg1.content)

```json
{
  "Seiten": [6, 7, 8, 28, 29, 30, 31, 32, 33]
}
```


In [43]:
# A.2) Ein Thread – Inhalt extrahieren basierend auf relevante Seiten – Bestest Ergebnis
msg1_content = [6, 7, 8, 26, 27, 28, 29, 30, 31, 32, 33]
relevant_pages = list(map(lambda index: pdf_prompts[index-1], msg1_content))
print(len(relevant_pages))

msg2 = model.invoke([
        AIMessage(
            content=[
                {
                    "type": "text",
                    "text": "Du bist ein Assistent zur getreuen Wiedergabe von Informationen aus einem PDF Dokument. Achte auf Vollständigkeit."
                },
            ]
        ),
        HumanMessage(
            content=[
                {
                    "type": "text",
                    "text": 'Extrahiere alle Informationen zu den folgenden Themen: Art der baulichen Nutzung, Maß der baulichen Nutzung, Bauweise und überbaubare Grundstücksfläche.'
                },
                *relevant_pages,
            ]
        ),
    ])
print(msg2.content)

11
Hier sind die extrahierten Informationen zu den Themen "Art der baulichen Nutzung", "Maß der baulichen Nutzung", "Bauweise" und "überbaubare Grundstücksfläche" aus dem Dokument:

### Art der baulichen Nutzung (§ 4)
1. **Allgemeines Wohngebiet (WA)**
   - Zulässig:
     - Wohngebäude
     - Läden, Schank- und Speisewirtschaften sowie nicht störende Handwerksbetriebe zur Versorgung des Gebiets
     - Anlagen für kirchliche, kulturelle, soziale, gesundheitliche und sportliche Zwecke
   - Nicht zulässig:
     - Betriebe des Beherbergungsgewerbes
     - Sonstige nicht störende Gewerbebetriebe
     - Anlagen für Verwaltungen
     - Gartenbaubetriebe
     - Tankstellen
   - Fremdwerbeanlagen sind im gesamten Plangebiet nicht zulässig.

### Maß der baulichen Nutzung (§ 5)
1. **Festsetzung in den Nutzungsschablonen der Planzeichnung (Teil A)**
2. **Zulässige Grundflächenzahl (GRZ) in den allgemeinen Wohngebieten:**
   - WA1: GRZ maximal 0,65
   - WA2: GRZ maximal 0,80
   - WA3: GRZ maximal 0

In [16]:
# B) Mehrere Threads
# Idee: Jede Seite einzeln verarbeiten, um Context-Window klein zu halten.
# Problem: Informationen sind auf mehrere Seiten verteilt => fehlender Kontext => unvollständige Resultate.
async def summarize(pdf_prompt):
    msg = model.invoke([
        AIMessage(
            content=[
                {
                    "type": "text",
                    "text": "Du bist ein Assistent um Textinformationen aus dem schriftlichen Teil eines Bebauungsplans zu extrahieren."
                },
            ]
        ),
        HumanMessage(
            content=[
            {
                "type": "text",
                "text": 'Extrahiere Informationen zu den folgenden Themen: Art der baulichen Nutzung, Maß der baulichen Nutzung, Bauweise, überbaubare Grundstücksfläche. Steht keine relevante Information zur Verfügung gebe "-" aus.'
            },
            pdf_prompt
            ]
        ),
    ])
    return msg.content

prompt_chain = list(map(lambda pdf_prompt: summarize(pdf_prompt), pdf_prompts))
results = await asyncio.gather(*prompt_chain)

for result in results:
    print(result)

- Art der baulichen Nutzung: -
- Maß der baulichen Nutzung: -
- Bauweise: -
- überbaubare Grundstücksfläche: -
- **Art der baulichen Nutzung**: Eingeschränktes Gewerbegebiet (GEE) (§ 8 BauNVO i.V.m. § 1 (4) BauNVO)

- **Maß der baulichen Nutzung**: -

- **Bauweise**: -

- **Überbaubare Grundstücksfläche**: -
Hier sind die extrahierten Informationen:

- **Art der baulichen Nutzung**: -
- **Maß der baulichen Nutzung**: 
  - Grundflächenzahl (§ 19 BauNVO): siehe Einträge im Lageplan
  - Geschossflächenzahl (§ 20 BauNVO): siehe Einträge im Lageplan
  - Höhe der baulichen Anlagen (§ 18 BauNVO): siehe Einträge im Lageplan. Die Gebäudehöhe wird gemessen von der Erdgeschossfußbodenhöhe (EFH) bis zur Schnittkante zwischen Außenwand und Dachhaut bzw. bis zur Oberkante Firstziegel.
- **Bauweise**: -
- **Überbaubare Grundstücksfläche**: -
- Art der baulichen Nutzung: -

- Maß der baulichen Nutzung: -

- Bauweise: 
  - Abweichende Bauweise. Es gilt die offene Bauweise, jedoch sind die Gebäudelängen